In [1]:
!pip install botbuilder-core aiohttp

In [3]:
!pip install botbuilder

  Preparing metadata (setup.py) ... - \ done
  Created wheel for botbuilder: filename=botbuilder-0.0.1-py3-none-any.whl size=1737 sha256=84ec552d1547e43e25759b274c05bc2f3b7ba2e75751e349694a2e89e3620a43
  Stored in directory: /home/azureuser/.cache/pip/wheels/fb/f8/4b/267974aea56612f68eb7b44c60e5978aac0319b11841b0167d
Successfully built botbuilder


In [7]:
!pip install botbuilder-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.5/244.5 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.10.1
    Uninstalling aiohttp-3.10.1:
      Successfully uninstalled aiohttp-3.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-automl-dnn-nlp 1.57.0 requires torch==1.13.1, but you have torch 2.4.1 which is incompatible.
adlfs 2024.7.0 requires fsspec>=2023.12.0, but you have fsspec 2023.10.0 which is incompatible.


In [9]:
!pip show botbuilder-ai
!pip show aiohttp
!pip show azure-cognitiveservices-language-luis


Name: botbuilder-ai
Version: 4.16.2
Summary: Microsoft Bot Framework Bot Builder
Home-page: https://www.github.com/Microsoft/botbuilder-python
Author: Microsoft
Author-email: 
License: MIT
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: aiohttp, aiounittest, azure-cognitiveservices-language-luis, botbuilder-core, botbuilder-schema
Required-by: 
Name: aiohttp
Version: 3.10.5
Summary: Async http client/server framework (asyncio)
Home-page: https://github.com/aio-libs/aiohttp
Author: 
Author-email: 
License: Apache 2
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: aiohappyeyeballs, aiosignal, async-timeout, attrs, frozenlist, multidict, yarl
Required-by: adlfs, aiohttp-cors, botbuilder-ai, datasets
Name: azure-cognitiveservices-language-luis
Version: 0.2.0
Summary: Microsoft Azure Cognitive Services LUIS Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: az

In [11]:
import json
import requests
from botbuilder.core import ActivityHandler, MessageFactory
from botbuilder.schema import ChannelAccount

class MyBot(ActivityHandler):
    def __init__(self):
        self.scoring_uri = "http://8877b5ca-e8cc-4c8c-89de-cd70256576f4.eastus2.azurecontainer.io/score"

    async def on_message_activity(self, turn_context):
        user_input = turn_context.activity.text
        
        # Prepare payload and headers
        input_data = {"input_text": user_input}
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}" if self.api_key else None
        }
        
        # Send request to the model
        try:
            response = requests.post(self.scoring_uri, headers=headers, data=json.dumps(input_data))
            if response.status_code == 200:
                model_response = response.json().get("response", "Sorry, I didn't understand that.")
            else:
                model_response = f"Error {response.status_code}: {response.text}"
        except Exception as e:
            model_response = f"Request failed: {e}"
        
        # Send model response back to user
        await turn_context.send_activity(MessageFactory.text(model_response))
